# Advanced Mapping
We use the geocoding capabilities locating areas of interest, and a map widget callback for solving various walking scenarios.

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry.functions import project
import ipywidgets as widgets
from ipywidgets import HBox, Label, Layout
from geopedestrian.categories import CategoryRegistry
from geopedestrian.services import drive_from, solve_walking, walk_from, walk_to
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory

In [2]:
host = 'geopedestrian.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

In [3]:
gis = GIS()

In [4]:
def radio_eventhandler(change):
    # Unregister callbacks
    map_view.on_click(solve_walking_areas, remove=True)
    map_view.on_click(find_places, remove=True)
    
    if 'Map' == change.new:
        pass
    elif 'Solve' == change.new:
        # Register callback
        map_view.on_click(solve_walking_areas)
    elif 'Find' == change.new:
        # Register callback
        map_view.on_click(find_places)

In [5]:
def solve_walking_areas(callback_map, geometry):
    """
    Solve walking areas which are accessible by pedestrians.
    """
    try:
        callback_map.remove_layers()
        callback_map.clear_graphics()
        callback_map.draw(geometry)        
        
        spatial_reference = geometry['spatialReference']
        in_sr = spatial_reference['wkid']
        out_sr = 4326
        project_result = project([geometry], in_sr, out_sr)
        location = project_result[0]
        lat, lon = location.y, location.x
        
        # Solve walking areas
        walking_result = solve_walking(client, lat, lon)
        walking_areas_sdf = FeatureSet.from_geojson(walking_result).sdf
        walking_areas_sdf.spatial.plot(callback_map, renderer_type='u', col='ToBreak', cmap='RdYlGn', alpha=0.35)
        
    except Exception as err:
        print(err)

def find_places(callback_map, geometry):
    """
    Find places which are accessible by pedestrians.
    """
    try:
        callback_map.remove_layers()
        callback_map.clear_graphics()
        callback_map.draw(geometry)        
        
        spatial_reference = geometry['spatialReference']
        in_sr = spatial_reference['wkid']
        out_sr = 4326
        project_result = project([geometry], in_sr, out_sr)
        location = project_result[0]
        lat, lon = location.y, location.x
        
        # Solve walking to the places category
        category_registry = CategoryRegistry()
        category = category_registry.find_id(category_widget.value)
        walking_result = walk_to(client, lat, lon, category)
        places_geojson = walking_result['places']
        places_sdf = FeatureSet.from_geojson(places_geojson).sdf
        places_sdf.spatial.plot(callback_map)
        
        routes_geojson = walking_result['routes']
        routes_sdf = FeatureSet.from_geojson(routes_geojson).sdf
        routes_sdf.spatial.plot(callback_map, renderer_type='s', palette=[0, 120, 0, 130], line_width=5)
    except Exception as err:
        print(err)

In [6]:
maptool_widget = widgets.RadioButtons(
    options=['Map', 'Solve', 'Find'],
    description='Tool:',
    disabled=False
)
location_widget = widgets.Text(
    value='San Diego Convention Center',
    placeholder='Name your area of interest',
    description='Location:',
    disabled=False   
)
category_widget = widgets.Dropdown(
    options=['Bar', 'Hotel'],
    value='Bar',
    description='Category:',
    disabled=False,
)
maptool_widget.observe(radio_eventhandler, names='value')
HBox([maptool_widget, location_widget, category_widget])

In [7]:
map_view = gis.map(location_widget.value)
map_view

MapView(layout=Layout(height='400px', width='100%'))